In [1]:
import numpy as np
from profiles import build_profile, build_kernel
from dispersion import SphericalJeansDispersion
from likelihood import GaussianLikelihood
from fitter import MinuitFitter
from utils import load_gaia

In [2]:
homedir = '/home/andrea/Desktop/work/DWARF/Jvalue/project1/test/Isotrop_Core_nonPlum'
MockSize = 100
dataSize = 100
dset = 1
# enter model choice - cf. casedir (options 1,2,3,4)
mod = 1
theta = 0.5
D = 100
data = load_gaia(homedir, MockSize, dataSize, dset, mod, D, True)

Using  /home/andrea/Desktop/work/DWARF/Jvalue/project1/test/Isotrop_Core_nonPlum/data/gs100_bs050_rcrs100_rarcinf_core_0400mpc3_df_100_1.dat


In [3]:
R = data[0]
v = data[1]
dv = data[2]
rh = data[-3]
r0_true = data[-2]
rho0_true = data[-1]

In [4]:
dm = build_profile('nfw')
st = build_profile('plummer', **{'rh':rh}) # non-Plummer Stellar profile
kr = build_kernel('iso') # isotropic kernel

In [5]:
dwarf_props = {'D':D, 'theta':theta, 'rt':np.inf, 'errs':False}
Sigma = SphericalJeansDispersion(dm, st, kr, dwarf_props)

In [6]:
LL = GaussianLikelihood([R, v, dv, 0], Sigma)

In [7]:
LL.free_pars.keys()

['J']

In [8]:
LL.set_free('dm_r0')
LL.set_free('dm_a')
LL.set_free('dm_b')
LL.set_free('dm_c')

In [9]:
global global_loglike
global_loglike = LL
M = MinuitFitter(LL)

In [11]:
M.set_minuit()

In [12]:
M.minuit.tol

0.1